In [1]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
import datetime

# 데이터 호출

In [44]:
url = '/content/Data_Train.xlsx'
df = pd.read_excel(url)
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [45]:
df.Airline.value_counts()

Jet Airways                          3849
IndiGo                               2053
Air India                            1752
Multiple carriers                    1196
SpiceJet                              818
Vistara                               479
Air Asia                              319
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: Airline, dtype: int64

In [49]:
air_count = df.Airline.unique()
air_count

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'Others', 'Vistara', 'Air Asia'], dtype=object)

# 데이터 전처리

## 1. Airline 전처리 _ dummies 필요

In [50]:
# count값에 맞춰 index 넘버링하기
air_count = df.Airline.value_counts().index

# 200 보다 적은 수의 airline은 Others 로 변환
airlist = [l for l in air_count if list(df.Airline).count(l) < 200]
df.Airline = df.Airline.replace(airlist, 'Others')
df.Airline.value_counts()

# 컬럼 미리 생성
df['Air_col'] = 0

In [51]:
# 200 보다 적은 수의 airline은 Others 로 변환
airlist = [l for l in air_count if list(df.Airline).count(l) < 200]
df.Airline = df.Airline.replace(airlist, 'Others')
df.Airline.value_counts()

Jet Airways          3849
IndiGo               2053
Air India            1752
Multiple carriers    1196
SpiceJet              818
Vistara               479
Air Asia              319
Others                217
Name: Airline, dtype: int64

In [52]:
# 컬럼 미리 생성
df['Air_col'] = 0

In [53]:
# Air_col 컬럼에 인덱스 번호로 넣기
for t in range(len(air_count)):
    df.loc[df.Airline == air_count[t], 'Air_col'] = t

print(df.Air_col.value_counts())
print(df.head())

0    3849
1    2053
2    1752
3    1196
4     818
5     479
6     319
7     217
Name: Air_col, dtype: int64
       Airline Date_of_Journey    Source Destination                  Route  \
0       IndiGo      24/03/2019  Banglore   New Delhi              BLR → DEL   
1    Air India       1/05/2019   Kolkata    Banglore  CCU → IXR → BBI → BLR   
2  Jet Airways       9/06/2019     Delhi      Cochin  DEL → LKO → BOM → COK   
3       IndiGo      12/05/2019   Kolkata    Banglore        CCU → NAG → BLR   
4       IndiGo      01/03/2019  Banglore   New Delhi        BLR → NAG → DEL   

  Dep_Time  Arrival_Time Duration Total_Stops Additional_Info  Price  Air_col  
0    22:20  01:10 22 Mar   2h 50m    non-stop         No info   3897        1  
1    05:50         13:15   7h 25m     2 stops         No info   7662        2  
2    09:25  04:25 10 Jun      19h     2 stops         No info  13882        0  
3    18:05         23:30   5h 25m      1 stop         No info   6218        1  
4    16:50       

## 2. 출발시간, 도착시간 전처리

In [54]:
# 출발시간, 도착시간에서 시간 단위만 뽑기
df['Dep_hour'] = df.Dep_Time.str.extract('([0-9]+)\:')
df['Arrival_hour'] = df.Arrival_Time.str.extract('([0-9]+)\:')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
 11  Air_col          10683 non-null  int64 
 12  Dep_hour         10683 non-null  object
 13  Arrival_hour     10683 non-null  object
dtypes: int64(2), object(12)
memory usage: 1.1+ MB


In [55]:
# print(df.Dep_hour.unique())
# print()
# print(df.Arrival_hour.unique())
print(df.Dep_hour)
print()
print(df.Arrival_hour)


0        22
1        05
2        09
3        18
4        16
         ..
10678    19
10679    20
10680    08
10681    11
10682    10
Name: Dep_hour, Length: 10683, dtype: object

0        01
1        13
2        04
3        23
4        21
         ..
10678    22
10679    23
10680    11
10681    14
10682    19
Name: Arrival_hour, Length: 10683, dtype: object


In [56]:
df.Dep_hour.astype(int)

0        22
1         5
2         9
3        18
4        16
         ..
10678    19
10679    20
10680     8
10681    11
10682    10
Name: Dep_hour, Length: 10683, dtype: int64

In [57]:
# object 형식으로 생성되기 때문에 to_numeric 처리
# df.Dep_hour = pd.to_numeric(df['Dep_hour'])
df.Arrival_hour = pd.to_numeric(df['Arrival_hour'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
 11  Air_col          10683 non-null  int64 
 12  Dep_hour         10683 non-null  object
 13  Arrival_hour     10683 non-null  int64 
dtypes: int64(3), object(11)
memory usage: 1.1+ MB


In [59]:
df.Dep_hour.unique()

array(['22', '05', '09', '18', '16', '08', '11', '20', '21', '17', '14',
       '04', '07', '10', '15', '06', '19', '23', '13', '02', '12', '00',
       '01', '03'], dtype=object)

## 3. Route drop 처리

In [14]:
df.drop(columns='Route', inplace=True)
df.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Air_col,Dep_hour,Arrival_hour
0,IndiGo,24/03/2019,Banglore,New Delhi,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,1,22,1
1,Air India,1/05/2019,Kolkata,Banglore,05:50,13:15,7h 25m,2 stops,No info,7662,2,05,13
2,Jet Airways,9/06/2019,Delhi,Cochin,09:25,04:25 10 Jun,19h,2 stops,No info,13882,0,09,4
3,IndiGo,12/05/2019,Kolkata,Banglore,18:05,23:30,5h 25m,1 stop,No info,6218,1,18,23
4,IndiGo,01/03/2019,Banglore,New Delhi,16:50,21:35,4h 45m,1 stop,No info,13302,1,16,21


## 4. Additional_Info 데이터 전처리

In [41]:
# 3. Additional_Info 전처리
# 20 보다 적은 수의 Additional_Info Others 로 변환
add_count = df.Additional_Info.value_counts().index
additional_thing = [l for l in add_count if list(df.Additional_Info).count(l) < 20]
df.Additional_Info = df.Additional_Info.replace(additional_thing, 'Others')
df.Additional_Info.value_counts()

No info                         8345
In-flight meal not included     1982
No check-in baggage included     320
Others                            36
Name: Additional_Info, dtype: int64

In [42]:
# 컬럼 미리 생성
df['Add_col'] = 0
df.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Air_col,Dep_hour,Arrival_hour,Add_col,Duration_hour
0,IndiGo,THU,Banglore,New Delhi,22:20,01:10 22 Mar,2h 50m,0,No info,3897,0,22,1,0,NaN
1,Air India,SAT,Kolkata,Banglore,05:50,13:15,7h 25m,2,No info,7662,0,05,13,0,NaN
2,Jet Airways,SUN,Delhi,Cochin,09:25,04:25 10 Jun,19h,2,No info,13882,0,09,4,0,NaN
3,IndiGo,SAT,Kolkata,Banglore,18:05,23:30,5h 25m,1,No info,6218,0,18,23,0,NaN
4,IndiGo,THU,Banglore,New Delhi,16:50,21:35,4h 45m,1,No info,13302,0,16,21,0,NaN


In [18]:
add_count = df.Additional_Info.value_counts().index
add_count

Index(['No info', 'In-flight meal not included',
       'No check-in baggage included', 'Others'],
      dtype='object')

In [19]:
# Add_col 컬럼에 인덱스 번호로 넣기
for t in range(len(add_count)):
    df.loc[df.Additional_Info == add_count[t], 'Add_col'] = t

df.Add_col.value_counts()

0    8345
1    1982
2     320
3      36
Name: Add_col, dtype: int64

In [20]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Air_col,Dep_hour,Arrival_hour,Add_col
0,IndiGo,24/03/2019,Banglore,New Delhi,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,0,22,1,0
1,Air India,1/05/2019,Kolkata,Banglore,05:50,13:15,7h 25m,2 stops,No info,7662,0,05,13,0
2,Jet Airways,9/06/2019,Delhi,Cochin,09:25,04:25 10 Jun,19h,2 stops,No info,13882,0,09,4,0
3,IndiGo,12/05/2019,Kolkata,Banglore,18:05,23:30,5h 25m,1 stop,No info,6218,0,18,23,0
4,IndiGo,01/03/2019,Banglore,New Delhi,16:50,21:35,4h 45m,1 stop,No info,13302,0,16,21,0


## 5. Duration 전처리 - 아직 안됨

In [21]:
# df.Duration[1].str.extract('([A-Za-z]+)h')
for i in range(len(df.Duration)):
    df.Duration_hour[i] = df.Duration[i].split()[0].replace('h', '')

df.head()

AttributeError: ignored

In [ ]:
df.head()

In [22]:
df['Duration_hour'] = df.Duration.str.extract('([A-Za-z]+)h')
# df['Duration_minute'] = df.Duration.str.extract(' ([A-Za-z]+)m')

In [23]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Air_col,Dep_hour,Arrival_hour,Add_col,Duration_hour
0,IndiGo,24/03/2019,Banglore,New Delhi,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,0,22,1,0,NaN
1,Air India,1/05/2019,Kolkata,Banglore,05:50,13:15,7h 25m,2 stops,No info,7662,0,05,13,0,NaN
2,Jet Airways,9/06/2019,Delhi,Cochin,09:25,04:25 10 Jun,19h,2 stops,No info,13882,0,09,4,0,NaN
3,IndiGo,12/05/2019,Kolkata,Banglore,18:05,23:30,5h 25m,1 stop,No info,6218,0,18,23,0,NaN
4,IndiGo,01/03/2019,Banglore,New Delhi,16:50,21:35,4h 45m,1 stop,No info,13302,0,16,21,0,NaN


## 6. Total_Stops 전처리

In [27]:
df.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [28]:
df[df.Total_Stops.isna()]

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Air_col,Dep_hour,Arrival_hour,Add_col,Duration_hour
9039,Air India,6/05/2019,Delhi,Cochin,09:45,09:25 07 May,23h 40m,NaN,No info,7480,0,09,9,0,NaN


In [29]:
#@title Total_Stops 결측값 처리
# Total_Stops의 결측값을 처리하려고 같은 비행이 있었는지 확인하려 헀는데
# 동일한 비행기는 없었으며,
# stops의 수는 다음과 같음
df[(df.Source == 'Delhi') & (df.Destination == 'Cochin')
    & (df.Dep_hour == '09') & (df.Arrival_hour == 9)
    ].Total_Stops.value_counts()

2 stops    14
1 stop     12
Name: Total_Stops, dtype: int64

In [30]:
# 혹시 걸린 시간이 차이날까 싶어 Duration을 확인해봤지만
# 모두 23h 40m 
df[(df.Source == 'Delhi') & (df.Destination == 'Cochin')
    & (df.Dep_hour == '09') & (df.Arrival_hour == 9)
    ].Duration.unique()

array(['23h 40m'], dtype=object)

In [31]:
# 결론 : 전반적으로 1 stop이 절반 이상이므로 1 stop 으로 처리
df.loc[df.Total_Stops.isna(),'Total_Stops'] = '1 stop'
df[df.Total_Stops.isna()]

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Air_col,Dep_hour,Arrival_hour,Add_col,Duration_hour


In [32]:
df.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', '4 stops'],
      dtype=object)

In [33]:
# 각 stop에 맞춘 
def handle_stops(x):
    # if type(x) == float: return np.NaN
    if x == 'non-stop': return 0
    return int(x.split()[0])

df.Total_Stops.apply(handle_stops)

0        0
1        2
2        2
3        1
4        1
        ..
10678    0
10679    0
10680    0
10681    0
10682    2
Name: Total_Stops, Length: 10683, dtype: int64

In [34]:
df.Total_Stops = df.Total_Stops.apply(handle_stops)
df.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Air_col,Dep_hour,Arrival_hour,Add_col,Duration_hour
0,IndiGo,24/03/2019,Banglore,New Delhi,22:20,01:10 22 Mar,2h 50m,0,No info,3897,0,22,1,0,NaN
1,Air India,1/05/2019,Kolkata,Banglore,05:50,13:15,7h 25m,2,No info,7662,0,05,13,0,NaN
2,Jet Airways,9/06/2019,Delhi,Cochin,09:25,04:25 10 Jun,19h,2,No info,13882,0,09,4,0,NaN
3,IndiGo,12/05/2019,Kolkata,Banglore,18:05,23:30,5h 25m,1,No info,6218,0,18,23,0,NaN
4,IndiGo,01/03/2019,Banglore,New Delhi,16:50,21:35,4h 45m,1,No info,13302,0,16,21,0,NaN


## 7. Date_of_Journey 전처리 _ dummies 필요



In [35]:
import datetime

days = ['MON', 'TUE', 'WEN', 'THU', 'FRI', 'SAT', 'SUN']

# print(days[df.Date_of_Journey.weekday()])


In [36]:
def chg_date(txt):
  return days[int(txt.split('/')[1])]

df.Date_of_Journey = df.Date_of_Journey.apply(chg_date)
df.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Air_col,Dep_hour,Arrival_hour,Add_col,Duration_hour
0,IndiGo,THU,Banglore,New Delhi,22:20,01:10 22 Mar,2h 50m,0,No info,3897,0,22,1,0,NaN
1,Air India,SAT,Kolkata,Banglore,05:50,13:15,7h 25m,2,No info,7662,0,05,13,0,NaN
2,Jet Airways,SUN,Delhi,Cochin,09:25,04:25 10 Jun,19h,2,No info,13882,0,09,4,0,NaN
3,IndiGo,SAT,Kolkata,Banglore,18:05,23:30,5h 25m,1,No info,6218,0,18,23,0,NaN
4,IndiGo,THU,Banglore,New Delhi,16:50,21:35,4h 45m,1,No info,13302,0,16,21,0,NaN


In [37]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Air_col,Dep_hour,Arrival_hour,Add_col,Duration_hour
0,IndiGo,THU,Banglore,New Delhi,22:20,01:10 22 Mar,2h 50m,0,No info,3897,0,22,1,0,NaN
1,Air India,SAT,Kolkata,Banglore,05:50,13:15,7h 25m,2,No info,7662,0,05,13,0,NaN
2,Jet Airways,SUN,Delhi,Cochin,09:25,04:25 10 Jun,19h,2,No info,13882,0,09,4,0,NaN
3,IndiGo,SAT,Kolkata,Banglore,18:05,23:30,5h 25m,1,No info,6218,0,18,23,0,NaN
4,IndiGo,THU,Banglore,New Delhi,16:50,21:35,4h 45m,1,No info,13302,0,16,21,0,NaN


In [40]:
df.Additional_Info

0        No info
1        No info
2        No info
3        No info
4        No info
          ...   
10678    No info
10679    No info
10680    No info
10681    No info
10682    No info
Name: Additional_Info, Length: 10683, dtype: object

# 중간점검
'Source', 'Destination', 'Duration',

In [46]:
print(df.columns)

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Dep_Time',
       'Arrival_Time', 'Duration', 'Total_Stops', 'Additional_Info', 'Price',
       'Air_col', 'Dep_hour', 'Arrival_hour', 'Add_col', 'Duration_hour'],
      dtype='object')
